In [44]:
import pandas as pd 
import json
import re
import psycopg2
import nltk
import nan 
from nltk.corpus import stopwords
from configparser import ConfigParser
from sklearn.feature_extraction.text import TfidfVectorizer

stopwords = set(stopwords.words('english'))

In [60]:
#This function facilitates the connection to the database
def connect():
    try:
        
        parser = ConfigParser()
        parser.read('database.ini')

        #params = config()
        host = parser.get('postgresql','DB_HOST')
        port = parser.get('postgresql','DB_PORT')
        database =parser.get('postgresql','DB_NAME')
        user = parser.get('postgresql','DB_USER')
        password = parser.get('postgresql','DB_PW')

        #params = config()
        #print(host, port, password, user, database)
        conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

        curr = conn.cursor()
        
#         query = """SELECT table_name from information_schema.tables WHERE table_schema = 'public'"""
#         curr.execute(query)
        
#         rows = curr.fetchall()

#         for row in rows:
#             print(row[0])

        return curr, conn
    except( Exception, psycopg2.DatabaseError) as err:
        print(err)
    
cur, con = connect()

In [ ]:
def remove_columns_from_products(table):
    #table.drop(['image', 'link', 'price', 'availability', 'retailer', 'details'], axis=1, inplace=True)
    table.dropna(inplace=True)

In [61]:
def get_wishlist(cur, con):
    query = "SELECT * FROM watchlist_cpu"
    cpus = pd.read_sql_query(query, con)
    cpus = pd.DataFrame(cpus)
    
    query = "SELECT * FROM watchlist_gpu"
    gpus = pd.read_sql_query(query, con)
    
    table = cpus.append(gpus)
    #any preprocessing needing to be done
    #table = remove_columns_from_products(table)
    table.drop_duplicates(subset='product_id', keep=False, inplace=True)

    return table
cur, con = connect()
get_wishlist(cur, con)

could not translate host name "siliconscraper-1.csvt0txubrxm.us-east-2.rds.amazonaws.com" to address: Unknown host



TypeError: cannot unpack non-iterable NoneType object

In [62]:
def get_all_products(cur, con):
    query = "SELECT id, brand, retailer, type, description FROM cpus"
    cpus = pd.read_sql_query(query, con)
    cpus = pd.DataFrame(cpus)
    
    query = "SELECT id, brand, retailer, type, description FROM gpus"
    gpus = pd.read_sql_query(query, con)
    
    table = cpus.append(gpus)
    
    # any preprocessing needed to be done
    table['description_cl'] = table['description'].apply(clean_data)
    table.drop(['description'], axis=1, inplace=True)
    table.dropna(inplace=True)

    return table

SyntaxError: unexpected EOF while parsing (<ipython-input-62-fefe4e811369>, line 1)

In [ ]:
#remove all products that are already in a wishlist 
#all_products = all_products.loc[all_products[all_products['id'] != wishlist_products['id']]]


In [ ]:
#rec = pd.Series(all_products.index)

#function to carry through the recommending 
def recommend(idx, cosine = similarity):
    
    #find the index of the product in the index series
    item = rec[rec == idx].index[0]
    
    #get array of similarities and sort them in descending order
    similarities = list(enumerate(similarity[item]))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    
    #get the top 5 products, excluding the product itself
    similarities = similarities[1:6]
   
    #get index of top 5 
    index = [i[0] for i in similarities]
    
    #get the products using the the retrieved index
     
    prods = all_products.iloc[index]
    resul = []
    for i, row in prods.iterrows():
        
        resul.append(uuid.UUID(i))
    return resul

In [ ]:
def clean_data(text):
    if text == '' or not list(text) or text == {} or text == '{}':
        return math.nan
    # text = json.loads(text)
    final = [(x +"-" + y).lower() for x,y in zip(list(text), list(text.values()))]
    return "  ".join(final)


In [ ]:
#get recommendations for all products in wishlist 
wishlist_cpu = wishlist_description[wishlist_description['type'] == 'cpu']
wishlist_gpu = wishlist_description[wishlist_description['type'] == 'gpu']

def get_recommendations(df_wishlist):
        recs = []
        
        wishlist = list(df_wishlist['product_id'])
        for i, product in enumerate(wishlist):
            response = list(recommend(product))
            recs.append(response)
       
        return recs
    
# cpus = get_recommendations(wishlist_cpu)
# gpus = get_recommendations(wishlist_gpu)

# result_cpu = pd.DataFrame(wishlist_cpu['id'], columns=['id','recommendations'])
# result_gpu = pd.DataFrame(wishlist_gpu['id'], columns=['id','recommendations'])

# result_cpu['recommendations'] = cpus
# result_gpu['recommendations'] = gpus

# result_cpu.set_index('id', inplace=True)
# result_gpu.set_index('id', inplace=True)

# result_gpu['recommendations']

In [ ]:
def get_all_product_recommendations(wishlist_description):
    temp = []
    global all_products
    all_products.set_index('product_id', inplace=True)
    wishlist_description.reset_index()
    for i, pro in all_products.iterrows():
        for j, wish in wishlist_description.iterrows():
            if i == wish['product_id']:
                temp.append((wish['product_id'], pro['type']))
                
    
  
    types = pd.DataFrame(temp, columns = ['product_id', 'type'])
    wishlist_cpu = types[types['type'] == 'cpu']#wishlist_description[wishlist_description['product_id'] == types['product_id'].item() & types['type'].item() == 'cpu']
    wishlist_gpu = types[types['type'] == 'gpu']#wishlist_description[wishlist_description['type'] == 'gpu']
   
    cpus = get_recommendations(wishlist_cpu)
    gpus = get_recommendations(wishlist_gpu)
    
    result_cpu = pd.DataFrame(wishlist_cpu['product_id'], columns=['id','recommendations'])
    result_gpu = pd.DataFrame(wishlist_gpu['product_id'], columns=['id','recommendations'])
    
    wishlist_gpu['recommendations'] = gpus
    wishlist_cpu['recommendations'] = cpus
    
    wishlist_cpu.set_index('product_id', inplace=True)
    wishlist_gpu.set_index('product_id', inplace=True)

    return wishlist_gpu, wishlist_cpu

In [ ]:
def update_gpu(gpu_recs):
    new_products = []
   
    try:

        for i, row in gpu_recs.iterrows():
            query = ("""SELECT id FROM recommendation_gpu WHERE id = %s""")
            cur.execute(query, (i,))
            product = cur.fetchone()
            
            if product is not None:
                 for recs in row['recommendations']:
                    query = (""" UPDATE recommendation_gpu SET recommended_id = %s WHERE product_id = %s""")
                    cur.execute(query, (recs, uuid.UUID(i)))
                    con.commit()
            else:
                new_products.append((i, row['recommendations']))
                
            
    except(psycopg2.DatabaseError) as err:
        print(err)
    finally:
        return new_products

In [ ]:
def update_cpu(cpu_recs):
    new_products = []
    try:
        for i, row in cpu_recs.iterrows():
            query = ("""SELECT product_id FROM recommendation_cpu WHERE product_id = %s""")
            cur.execute(query, (i,))
            product = cur.fetchone()

            if product is not None:
                for recs in row['recommendations']:
                    query = (""" UPDATE recommendation_cpu SET recommended_id = %s WHERE product_id = %s""")
                    cur.execute(query, (recs, uuid.UUID(i)))
                    con.commit()
            else:
                new_products.append((i, row['recommendations']))

            
    except(psycopg2.DatabaseError) as err:
        print(err)
    finally:
        return new_products

In [ ]:
def insert_cpu(id, cpu_arr):
    psycopg2.extras.register_uuid()
    for cpu in cpu_arr:
        query = ("""INSERT INTO recommendation_cpu (product_id, recommended_id) VALUES (%s, %s);""")
        cur.execute(query,(id, cpu))
        con.commit()


def insert_gpu(id, gpu_arr):
    psycopg2.extras.register_uuid()
    for gpu in gpu_arr:
        query = ("""INSERT INTO recommendation_gpu (product_id, recommended_id) VALUES (%s, %s);""")
        cur.execute(query,(id, gpu))
        con.commit()


In [ ]:
 wishlist_products = get_wishlist(cur, con)

global all_products
all_products = get_all_products(cur, con)


#all_products.set_index('id', inplace=True)
all_products.rename(columns={'id': 'product_id'}, inplace=True)
all_products['all_features'] = all_products['brand'] +" "+ all_products['retailer'] +" "+ all_products['description_cl']
all_products['all_features'] = all_products['all_features'].apply(lambda x: x.lower())

vectorizor = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0)
tfidf_matrix = vectorizor.fit_transform(all_products['all_features'])

global similarity
similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

global rec
rec = pd.Series(all_products['product_id'])

gpu_recs, cpu_recs = get_all_product_recommendations(wishlist_products)


new_gpu = update_gpu(gpu_recs)
new_cpu = update_cpu(cpu_recs)

if new_gpu is not None:
    df_gpu = pd.DataFrame(new_gpu, columns=['id', 'recs'])

    for i, row in df_gpu.iterrows():
        insert_gpu(row['id'], row['recs'])

if new_cpu is not None:
    df_cpu = pd.DataFrame(new_cpu, columns=['id', 'recs'])
    for i, row in df_cpu.iterrows():
        insert_cpu(row['id'], row['recs'])
con.close()


In [ ]:
def main():
    #get users
    cur, con = connect()
    users = get_users(cur, con)
    
    #get users wishlist; filter
    wishlist_products = get_wishlist(cur, con)
    
    #get all products that are not in wishlist; filter
    all_products = get_all_products(cur, con)
    #encode data 
    
    #predict